In [1]:
import awkward as ak
import numpy as np
import pandas as pd


import ehrapy as ep
import scanpy as sc
from anndata import AnnData
import mudata as md
from mudata import MuData
import os

/opt/anaconda3/envs/ehrapy_latents/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [211]:
import pandas as pd
import ehrapy as ep
import glob

class OMOP():
    
    def __init__(self, file_paths):
        self.base = file_paths
        file_list = glob.glob(os.path.join(file_paths, '*'))
        self.loaded_tabel = None
        self.tables = []
        for file_path in file_list:
            file_name = file_path.split('/')[-1].removesuffix('.csv')
            self.tables.append(file_name)
    
    # Some Plot functions to plot
    # Load important tables into AnnData
    # Time series data into .obsm as Awkward Array
    # Episode Level Data into .obs/.X
    def load(self, level='stay_level'):
        if level == 'stay_level':
            self.visit_occurrence = pd.read_csv(f'{self.base}/visit_occurrence.csv')
            self.person = pd.read_csv(f'{self.base}/person.csv', index_col='person_id')
            self.death = pd.read_csv(f'{self.base}/death.csv', index_col='person_id')
            self.measurement = pd.read_csv(f'{self.base}/measurement.csv')
            self.observation = pd.read_csv(f'{self.base}/observation.csv')
            self.drug_exposure = pd.read_csv(f'{self.base}/drug_exposure.csv')

            self.loaded_tabel = ['visit_occurrence', 'person', 'death', 'measurement', 'observation', 'drug_exposure']
            self.joined_table = pd.merge(self.visit_occurrence, self.person, on='person_id', how='left')
            self.joined_table = pd.merge(self.visit_occurrence, self.death, on='person_id', how='left')
            


            obs_only_list = list(omop.joined_table.columns)
            obs_only_list.remove('visit_occurrence_id')
            adata = ep.ad.df_to_anndata(
                self.joined_table, index_column="visit_occurrence_id", columns_obs_only = obs_only_list)
            
            for column in self.measurement.columns:
                if column != 'visit_occurrence_id':
                    obs_list = []
                    for visit_occurrence_id in adata.obs.index:
                        obs_list.append(list(self.measurement[self.measurement['visit_occurrence_id'] == int(visit_occurrence_id)][column]))
                    adata.obsm[column]= ak.Array(obs_list)

            for column in self.drug_exposure.columns:
                if column != 'visit_occurrence_id':
                    obs_list = []
                    for visit_occurrence_id in adata.obs.index:
                        obs_list.append(list(self.drug_exposure[self.drug_exposure['visit_occurrence_id'] == int(visit_occurrence_id)][column]))
                    adata.obsm[column]= ak.Array(obs_list)
            
            for column in self.observation.columns:
                if column != 'visit_occurrence_id':
                    obs_list = []
                    for visit_occurrence_id in adata.obs.index:
                        obs_list.append(list(self.observation[self.observation['visit_occurrence_id'] == int(visit_occurrence_id)][column]))
                    adata.obsm[column]= ak.Array(obs_list)
            
        
        return adata


2023-11-20 18:06:28,994 - root INFO - Transformed passed dataframe into an AnnData object with n_obs x n_vars = `852` x `0`.


In [ ]:
# Using MIMIC-IV demo data in the OMOP Common Data Model
# https://physionet.org/content/mimic-iv-demo-omop/0.9/
omop = OMOP("/Users/xinyuezhang/mimic-iv-demo-data-in-the-omop-common-data-model-0.9/1_omop_data_csv")
adata = omop.load('stay_level')

In [210]:
omop.tables

['provider',
 '2b_vocabulary',
 'observation_period',
 'drug_exposure',
 'care_site',
 '2b_concept',
 'cohort_definition',
 'cost',
 'specimen',
 'death',
 'device_exposure',
 'attribute_definition',
 'fact_relationship',
 'dose_era',
 'location',
 'measurement',
 'cohort',
 'note_nlp',
 'cohort_attribute',
 'condition_occurrence',
 '2b_concept_relationship',
 'drug_era',
 'cdm_source',
 'visit_occurrence',
 'metadata',
 'person',
 'observation',
 'visit_detail',
 'payer_plan_period',
 'procedure_occurrence',
 'note',
 'condition_era']

In [212]:
omop.loaded_tabel

['visit_occurrence',
 'person',
 'death',
 'measurement',
 'observation',
 'drug_exposure']

In [192]:
adata

AnnData object with n_obs × n_vars = 852 × 0
    obs: 'person_id', 'visit_concept_id', 'visit_start_date', 'visit_start_datetime', 'visit_end_date', 'visit_end_datetime', 'visit_type_concept_id', 'provider_id', 'care_site_id', 'visit_source_value', 'visit_source_concept_id', 'admitting_source_concept_id', 'admitting_source_value', 'discharge_to_concept_id', 'discharge_to_source_value', 'preceding_visit_occurrence_id', 'death_date', 'death_datetime', 'death_type_concept_id', 'cause_concept_id', 'cause_source_value', 'cause_source_concept_id'
    uns: 'numerical_columns', 'non_numerical_columns'
    obsm: 'measurement_id', 'person_id', 'measurement_concept_id', 'measurement_date', 'measurement_datetime', 'measurement_time', 'measurement_type_concept_id', 'operator_concept_id', 'value_as_number', 'value_as_concept_id', 'unit_concept_id', 'range_low', 'range_high', 'provider_id', 'visit_detail_id', 'measurement_source_value', 'measurement_source_concept_id', 'unit_source_value', 'value_sou

In [ ]:
# Some setter and getter functions

# Some functions
# Define cohort

# Plot functions to: 
# 1) plot time series feature(s) of one patients or mutiple patients
# 2) plot data density of time series features

# More functions:
# Subetting along columns: support extract certain features from certain tables and save in anndata. (E.g. heart rate from measurement table. )
# Subetting along rows: support data subsetting based on certain diease label / time / … (E.g. extract First 48-hour data since admission.)
# get the aggregation/representation of time series features, and put into .X (min/max/median/mean) .uns/.obsm (representation).
# Query, Map concept in concept table.

In [209]:
len(adata.obsm['measurement_id'])

In [65]:
import yaml
with open("/Users/xinyuezhang/MedFuse-main/mimic4extract/mimic3benchmark/resources/icd_9_10_definitions_2.yaml") as definitions_file:
    definitions = yaml.safe_load(definitions_file)

code_to_group = {}


for group in definitions:
    codes = definitions[group]['codes']
    for code in codes:
        if code not in code_to_group:
            code_to_group[code] = group
        else:
            print(f'code, {code}')
            assert code_to_group[code] == group

code, I6359
code, I669
code, I2109
code, I2109
code, I2109
code, I2109
code, I2109
code, I2119
code, I2119
code, I2111
code, I2111
code, I2119
code, I2119
code, I2119
code, I2129
code, I2129
code, I2129
code, I2129
code, I2129
code, I214
code, I214
code, I2129
code, I2129
code, I2129
code, I21A9
code, I21A9
code, R001
code, Z4931
code, Z4932
code, J439
code, J449
code, G9782
code, G9782
code, J9589
code, K9189
code, T8172XA
code, T888XXA
code, T814XXA
code, T8189XA
code, I441
code, I452
code, I452
code, I456
code, I509
code, I2510
code, I259
code, E0965
code, E0921
code, E1310
code, E1310
code, E1065
code, E1165
code, E1069
code, E1165
code, E1065
code, E1165
code, E1065
code, E1165
code, E1065
code, E1165
code, E1065
code, E1165
code, E1065
code, E1069
code, E1169
code, E1069
code, E118
code, E108
code, Z4681
code, K56699
code, K56699
code, K56699
code, K56699
code, K56699
code, K56699
code, K56699
code, K270
code, K272
code, K274
code, K276
code, K280
code, K282
code, K284
code, K286